# Coursera Capstone Project

This notebook will be used for the capstone project.

### Criteria

For **first week**, you will required to submit the following:
* A description of the problem and a discussion of the background. (15 marks)
* A description of the data and how it will be used to solve the problem. (15 marks)

For the **second week**, the final deliverables of the project will be:
* A link to your Notebook on your Github repository, showing your code. (15 marks)
* A full report consisting of all of the following components (15 marks):
    * Introduction where you discuss the business problem and who would be interested in this project.
    * Data where you describe the data that will be used to solve the problem and the source of the data.
    * Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.
    * Results section where you discuss the results.
    * Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.
    * Conclusion section where you conclude the report.
* Your choice of a presentation or blogpost. (10 marks)

In [1]:
# Importing libraries

import pandas as pd
import numpy as np
import random
import requests

# module to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import json

# import tools for webscraping
from bs4 import BeautifulSoup
from urllib.request import urlopen

import folium

## 1. Problem Description and Background Discussion

### 1.1 Problem Description
Comparing Central New York to Central Toronto. How 
### 1.2 Background Discussion
The Background

## 2. Data Description 

### 2.1 Description of Data and Data Source
Data ...

### 2.2 How will the Data be used to solve the Problem
...


In [2]:
import urllib
import json

# Get the dataset metadata by passing package_id to the package_search endpoint
# For example, to retrieve the metadata for this dataset:

url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/package_show"
params = { "id": "9edb9628-1213-42bd-8352-5c4ed28e9e42"}
response = urllib.request.urlopen(url, data=bytes(json.dumps(params), encoding="utf-8"))
package = json.loads(response.read())
print(package["result"])

# Get the data by passing the resource_id to the datastore_search endpoint
# See https://docs.ckan.org/en/latest/maintaining/datastore.html for detailed parameters options
# For example, to retrieve the data content for the first resource in the datastore:

for idx, resource in enumerate(package["result"]["resources"]):
    if resource["datastore_active"]:
        url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/datastore_search"
        p = { "id": resource["id"] }
        r = urllib.request.urlopen(url, data=bytes(json.dumps(p), encoding="utf-8"))
        data = json.loads(r.read())
        df = pd.DataFrame(data["result"]["records"])
        break
df

{'license_title': 'Open Government Licence – Toronto', 'owner_unit': None, 'relationships_as_object': [], 'topics': 'Business,Locations and mapping', 'owner_email': 'opendata@toronto.ca', 'excerpt': 'Boundaries for the active Business Improvement Areas within the City of Toronto.', 'private': False, 'owner_division': 'Economic Development & Culture', 'num_tags': 8, 'id': '9edb9628-1213-42bd-8352-5c4ed28e9e42', 'metadata_created': '2019-07-23T16:45:34.043754', 'refresh_rate': 'Weekly', 'title': 'Business Improvement Areas', 'license_url': 'https://open.toronto.ca/open-data-license/', 'state': 'active', 'information_url': None, 'license_id': 'open-government-licence-toronto', 'type': 'dataset', 'resources': [{'cache_last_updated': None, 'package_id': '9edb9628-1213-42bd-8352-5c4ed28e9e42', 'datastore_active': True, 'id': 'd173e644-ace0-45e0-be43-8ba02fb116eb', 'size': None, 'format': 'GeoJSON', 'state': 'active', 'hash': '', 'description': '', 'is_preview': True, 'last_modified': '2020-0

,_id,AREA_ID,DATE_EFFECTIVE,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry
0,3162,2481845,2020-02-04T17:20:36,26006945,None,059-00,059-00,College Promenade,College Promenade,310882.714,4834896.895,-79.424591,43.653991,17569265,1.211422e+05,2553.921411,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4203..."
1,3163,2481844,2020-02-04T17:20:36,26006944,None,109-01,109-01,CityPlace and Fort York,CityPlace and Fort York,312780.016,4833181.751,-79.401095,43.638534,17569281,1.192955e+06,5327.234526,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4103..."
2,3164,2481843,2020-02-04T17:20:36,26006943,None,065-00,065-00,Chinatown,Chinatown,313028.145,4834884.119,-79.397994,43.653855,17569297,2.932214e+05,4749.595430,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4006..."
3,3165,2481842,2020-02-04T17:20:36,26006942,None,012-02,012-02,Cabbagetown,Cabbagetown,315349.971,4836048.305,-79.369187,43.664305,17569313,3.156133e+05,5933.750217,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[-7..."
4,3166,2481841,2020-02-04T17:20:36,26006941,None,027-00,027-00,Broadview Danforth,Broadview Danforth,316338.822,4837427.069,-79.356896,43.676701,17569329,1.940808e+05,3085.887431,"{""type"": ""Polygon"", ""coordinates"": [[[-79.3528..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,3210,2481797,2020-02-04T17:20:36,26006897,None,063-02,063-02,Bayview Leaside,Bayview Leaside,314858.259,4840661.115,-79.375197,43.705833,17570033,1.124386e+05,2242.427742,"{""type"": ""Polygon"", ""coordinates"": [[[-79.3761..."
79,3211,2481796,2020-02-04T17:20:36,26006896,None,062-01,062-01,Albion Islington Square,Albion Islington Square,299679.981,4844059.783,-79.563561,43.736476,17570049,3.126040e+05,3979.632937,"{""type"": ""Polygon"", ""coordinates"": [[[-79.5652..."
80,3212,2481795,2020-02-04T17:20:36,26006895,None,085-02,085-02,Baby Point Gates,Baby Point Gates,305862.925,4835307.323,-79.486822,43.657709,17570065,1.505034e+05,2820.261946,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4886..."
81,3213,2481794,2020-02-04T17:20:36,26006894,None,064-00,064-00,Bloor Street,Bloor Street,313798.466,4836678.158,-79.388414,43.669995,17570081,3.659230e+05,4757.401630,"{""type"": ""Polygon"", ""coordinates"": [[[-79.3948..."


### API for accessing foursquare data

In [9]:
CLIENT_ID = '5MEM4YM205NTQBOMWUQX00NHLMW2GJGAV2OPGIHK55JSJKFU' # your Foursquare ID
CLIENT_SECRET = 'XQ34UGNCZTPZWQFKCIVSYLXHK533UR24OSHJ1BKLE2SSZTT3' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5MEM4YM205NTQBOMWUQX00NHLMW2GJGAV2OPGIHK55JSJKFU
CLIENT_SECRET:XQ34UGNCZTPZWQFKCIVSYLXHK533UR24OSHJ1BKLE2SSZTT3
